In [ ]:
!pip install osmnx

In [45]:
import pandas as pd
import numpy as np
import osmnx as ox

In [46]:
#Define Haversine class for calculating shortest distance between accident position and "nodes" which extracted using "osmnx" library.
import math
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1 = math.radians(lon1)
    lat1 = math.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)
    # lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6378 # Radius of earth in kilometers
    return c * r

In [47]:
G = ox.graph_from_place('Los Angeles, California', network_type='drive')

In [4]:
!pip install pandas --upgrade

Requirement already up-to-date: pandas in /Users/lyudonghang/opt/anaconda3/envs/interface/lib/python3.7/site-packages (1.3.5)


In [48]:
data_path = "US_Accidents_Dec21_updated.csv"
data = pd.read_csv(data_path)
kaggle_LA = data[data["City"]=="Los Angeles"]
print(kaggle_LA.shape)
kaggle_LA['Start_Time'] =  pd.to_datetime(kaggle_LA['Start_Time'], format='%Y-%m-%d %H:%M:%S')
kaggle_LA['Year'] = pd.DatetimeIndex(kaggle_LA['Start_Time']).year
kaggle_LA_2020 = kaggle_LA[kaggle_LA["Year"]==2020]
print(kaggle_LA_2020.shape)
print(kaggle_LA_2020.head())

(68956, 47)
(16509, 48)
                ID  Severity          Start_Time             End_Time  \
1735387  A-1735388         2 2020-11-28 01:33:00  2020-11-28 03:13:16   
1735404  A-1735405         2 2020-11-10 02:32:00  2020-11-10 04:46:20   
1735563  A-1735564         2 2020-11-20 08:47:00  2020-11-20 12:17:14   
1735605  A-1735606         2 2020-12-22 12:38:30  2020-12-22 20:46:30   
1735606  A-1735607         2 2020-12-10 20:34:30  2020-12-10 21:33:30   

         Start_Lat   Start_Lng    End_Lat     End_Lng  Distance(mi)  \
1735387  33.963490 -118.370694  33.963061 -118.370409         0.034   
1735404  34.152964 -118.278776  34.153274 -118.277476         0.077   
1735563  34.051600 -118.458568  34.052739 -118.458171         0.082   
1735605  33.981910 -118.288154  33.982370 -118.281096         0.406   
1735606  34.135160 -118.358750  34.139809 -118.365350         0.496   

                                               Description  ...  Station  \
1735387  Incident on S LA CIENEGA 

/Users/lyudonghang/opt/anaconda3/envs/interface/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/lyudonghang/opt/anaconda3/envs/interface/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
import datetime
print(datetime.date(2020, 6, 23)<datetime.date(2020, 6, 24))

True


In [13]:
ts = pd.to_datetime("2020-03-30")+pd.DateOffset(days=10)
print(ts)

2020-04-09 00:00:00


In [49]:
import random
random.seed(0)
def addNegativeSamples(df):
    global balanced_kaggle_california_df
    #Specifying raw dataframe
    raw_df = df.copy()
    raw_df["Accident"] = 1
    
    #Specifying date dataframe
    negative_date_df = df.copy()
    negative_date_df["Accident"] = 0

    #Specifying hour dataframe
    negative_hour_df = df.copy()
    negative_hour_df["Accident"] = 0
    
    #Offset "DATE" column with -15~15 days without 0
    day_list = list(range(-15, 16))
    day_list.remove(0)
    negative_date_df["Start_Time"] = negative_date_df["Start_Time"] + pd.DateOffset(days=random.choice(day_list))

    #Offset "HOUR" column with -12~12 hour without 0
    hour_list = list(range(-12, 13))
    hour_list.remove(0)
    negative_hour_df["Start_Time"] = negative_hour_df["Start_Time"] + pd.DateOffset(hours=random.choice(hour_list))

    balanced_kaggle_california_df = pd.concat([raw_df, negative_date_df, negative_hour_df],ignore_index=True)
    return balanced_kaggle_california_df

kaggle_LA_2020 = addNegativeSamples(kaggle_LA_2020)
print(kaggle_LA_2020.shape)
kaggle_LA_2020['Hour'] = pd.DatetimeIndex(kaggle_LA_2020['Start_Time']).hour
kaggle_LA_2020['Date'] = kaggle_LA_2020['Start_Time'].apply(lambda x: str(x).split()[0])
print(kaggle_LA_2020.shape)
print(kaggle_LA_2020.head())

(49527, 49)
(49527, 51)
          ID  Severity          Start_Time             End_Time  Start_Lat  \
0  A-1735388         2 2020-11-28 01:33:00  2020-11-28 03:13:16  33.963490   
1  A-1735405         2 2020-11-10 02:32:00  2020-11-10 04:46:20  34.152964   
2  A-1735564         2 2020-11-20 08:47:00  2020-11-20 12:17:14  34.051600   
3  A-1735606         2 2020-12-22 12:38:30  2020-12-22 20:46:30  33.981910   
4  A-1735607         2 2020-12-10 20:34:30  2020-12-10 21:33:30  34.135160   

    Start_Lng    End_Lat     End_Lng  Distance(mi)  \
0 -118.370694  33.963061 -118.370409         0.034   
1 -118.278776  34.153274 -118.277476         0.077   
2 -118.458568  34.052739 -118.458171         0.082   
3 -118.288154  33.982370 -118.281096         0.406   
4 -118.358750  34.139809 -118.365350         0.496   

                                         Description  ...  Traffic_Signal  \
0  Incident on S LA CIENEGA BLVD near W FLORENCE ...  ...            True   
1                           

In [50]:
# convert hour, day and month to take each one into corresponding category
def convert_time(df):
    def convert_hour(x):
        if 0<=x and x<=6:
            return 0
        elif 7<=x and x<=8:
            return 1
        elif 9<=x and x<=11:
            return 2
        elif 12<=x and x<=13:
            return 3
        elif 14<=x and x<=16:
            return 4
        elif 17<=x and x<=19:
            return 5
        else:
            return 6
        
    def convert_month(x):
        year, month, day = x.split("-")
        year = int(year)
        month = int(month)
        day = int(day)
        current_date = datetime.date(year, month, day)
        if current_date>=datetime.date(year, 3, 20) and current_date<datetime.date(year,6, 21):
            return 0
        elif current_date>=datetime.date(year, 6, 21) and current_date<datetime.date(year,9, 23):
            return 1
        elif current_date>=datetime.date(year, 9, 23) and current_date<datetime.date(year,12, 21):
            return 2
        else:
            return 3
    
    def convert_day(x):
        year, month, day = x.split("-")
        year = int(year)
        month = int(month)
        day = int(day)
        wk = datetime.date(year, month, day).weekday()
        if wk>=1 and wk<=4:
            return 0
        else:
            return 1
        
    df["converted_hour"] = df["Hour"].apply(lambda x: convert_hour(x))
    df["converted_month"] = df["Date"].apply(lambda x: convert_month(x))
    df["converted_day"] = df["Date"].apply(lambda x: convert_day(x))
    return df
kaggle_LA_2020 = convert_time(kaggle_LA_2020)
print(kaggle_LA_2020.shape)
print(kaggle_LA_2020.head())

(49527, 54)
          ID  Severity          Start_Time             End_Time  Start_Lat  \
0  A-1735388         2 2020-11-28 01:33:00  2020-11-28 03:13:16  33.963490   
1  A-1735405         2 2020-11-10 02:32:00  2020-11-10 04:46:20  34.152964   
2  A-1735564         2 2020-11-20 08:47:00  2020-11-20 12:17:14  34.051600   
3  A-1735606         2 2020-12-22 12:38:30  2020-12-22 20:46:30  33.981910   
4  A-1735607         2 2020-12-10 20:34:30  2020-12-10 21:33:30  34.135160   

    Start_Lng    End_Lat     End_Lng  Distance(mi)  \
0 -118.370694  33.963061 -118.370409         0.034   
1 -118.278776  34.153274 -118.277476         0.077   
2 -118.458568  34.052739 -118.458171         0.082   
3 -118.288154  33.982370 -118.281096         0.406   
4 -118.358750  34.139809 -118.365350         0.496   

                                         Description  ...  Civil_Twilight  \
0  Incident on S LA CIENEGA BLVD near W FLORENCE ...  ...           Night   
1                            EB 134 JWO 

In [51]:
kaggle_LA_2020["Lat"] = (kaggle_LA_2020["Start_Lat"]+kaggle_LA_2020["End_Lat"])/2
kaggle_LA_2020["Lng"] = (kaggle_LA_2020["Start_Lng"]+kaggle_LA_2020["End_Lng"])/2
kaggle_LA_2020_new = kaggle_LA_2020[["Lat", "Lng", "converted_hour", "converted_month", "converted_day", "Accident"]]
print(kaggle_LA_2020_new.shape)
print(kaggle_LA_2020_new.head())

(49527, 6)
         Lat         Lng  converted_hour  converted_month  converted_day  \
0  33.963276 -118.370552               0                2              1   
1  34.153119 -118.278126               0                2              0   
2  34.052170 -118.458370               1                2              0   
3  33.982140 -118.284625               3                3              0   
4  34.137484 -118.362050               6                2              0   

   Accident  
0         1  
1         1  
2         1  
3         1  
4         1  


In [60]:
nodes_list = sorted(list(G.nodes))
tmp = nodes_list[100]
print(G.nodes[tmp])

{'y': 34.1235604, 'x': -118.226157, 'highway': 'stop', 'street_count': 5}


In [53]:
temp = 0
for edge in G.edges:
    print(G[edge[0]][edge[1]][0])
    if temp>6:
        break
    temp+=1

{'osmid': [907145138, 895315641, 895315642, 398770659], 'lanes': '6', 'name': 'National Boulevard', 'highway': 'secondary', 'maxspeed': '35 mph', 'oneway': False, 'length': 88.746, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fdf9f970050>}
{'osmid': [398770658, 759468526, 759468527], 'oneway': False, 'lanes': '4', 'name': 'National Boulevard', 'highway': 'secondary', 'maxspeed': '35 mph', 'length': 102.25999999999999, 'tunnel': 'yes', 'geometry': <shapely.geometry.linestring.LineString object at 0x7fdf9f9701d0>}
{'osmid': 398771138, 'lanes': '5', 'name': 'National Boulevard', 'highway': 'secondary', 'maxspeed': '35 mph', 'oneway': False, 'length': 19.948999999999998, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fdf9f970090>}
{'osmid': [404964730, 398771139], 'lanes': ['4', '5'], 'name': 'National Boulevard', 'highway': 'secondary', 'maxspeed': '35 mph', 'oneway': False, 'length': 109.153, 'geometry': <shapely.geometry.linestring.LineString object

In [61]:
nodes_num = len(nodes_list)
print("nodes_num", nodes_num)
feature_matrix = np.zeros((nodes_num,5))
labels = []
times = 0
num_1s = 0
num_0s = 0
for node in nodes_list:
    lats = G.nodes[node]["y"]
    lngs = G.nodes[node]["x"]
    if "street_count" in G.nodes[node]:
        street_count = G.nodes[node]["street_count"]
    else:
        street_count = 0
    dist_min = float("inf")
    lat_accident = np.array(kaggle_LA_2020_new["Lat"])
    lng_accident = np.array(kaggle_LA_2020_new["Lng"])
    dist = haversine(lngs, lats, lng_accident, lat_accident)
    dist_tmp = np.min(dist)
    dist_indexes = list(np.where(dist==dist_tmp)[0])
    dist_index = random.choice(dist_indexes)
    if dist_tmp<dist_min:
        dist_min = dist_tmp
        hours = int(kaggle_LA_2020_new["converted_hour"].iloc[dist_index])
        months = int(kaggle_LA_2020_new["converted_month"].iloc[dist_index])
        days = int(kaggle_LA_2020_new["converted_day"].iloc[dist_index])
    feature_matrix[times, 0] = lats
    feature_matrix[times, 1] = lngs
    feature_matrix[times, 2] = hours
    feature_matrix[times, 3] = months
    feature_matrix[times, 4] = days
    labels.append(kaggle_LA_2020_new["Accident"].iloc[dist_index])
    times+=1
    if kaggle_LA_2020["Accident"].iloc[dist_index]==1:
        num_1s+=1
    else:
        num_0s+=1
print("number of 1s:", num_1s)
print("number of 0s:", num_0s)
        

nodes_num 49248
number of 1s: 16670
number of 0s: 32578


In [62]:
lat_min = np.min(feature_matrix[:, 0])
lat_max = np.max(feature_matrix[:, 0])
lng_min = np.min(feature_matrix[:, 1])
lng_max = np.max(feature_matrix[:, 1])
# 20 x 20 = 400 blocks
lat_interval = (lat_max-lat_min)/20
lng_interval = (lng_max-lng_min)/20
save_dict = {}
for i in range(feature_matrix.shape[0]):
    xx = min(int((feature_matrix[i, 0]-lat_min)/lat_interval), 19)
    yy = min(int((feature_matrix[i, 1]-lng_min)/lng_interval), 19)
    if tuple([xx, yy]) not in save_dict:
        save_dict[tuple([xx, yy])] = [i]
    else:
        save_dict[tuple([xx, yy])].append(i)

In [63]:
import torch
from torch_geometric.data import Data

graphs = []
tim = 0
for key in save_dict:
    indexes = save_dict[key]
    if len(indexes)<=5:
        continue
    node_features = []
    nodes = []
    node_labels = []
    for index in indexes:
        node_features.append(list(feature_matrix[index, 2:]))
        node_labels.append(labels[index])
        nodes.append(nodes_list[index])
    node_features = np.array(node_features)
    x = torch.tensor(node_features, dtype=torch.float)
    node_labels = np.array(node_labels)
    y = torch.tensor(node_labels)
    
    sub_G = G.subgraph(nodes)
    sources = []
    targets = []
    edge_attributes = []  # length, bridge, lanes, oneway, maxspeed, tunnel
    for edge in sub_G.edges:
        sources.append(edge[0])
        targets.append(edge[1])
        edge_attr = sub_G[edge[0]][edge[1]][0]
        if "length" in edge_attr:
            length = edge_attr["length"]
        else:
            length = 0
        
        if "bridge" in edge_attr:
            if edge_attr["bridge"]=="yes":
                bridge=1
            else:
                bridge=0
        else:
            bridge=0
            
        if "tunnel" in edge_attr:
            if edge_attr["tunnel"]=="yes":
                tunnel=1
            else:
                tunnel=0
        else:
            tunnel=0
        
        if "lanes" in edge_attr:
            if type(edge_attr["lanes"])==list:
                lanes = int(edge_attr["lanes"][0])
            else:
                lanes = int(edge_attr["lanes"])
        else:
            lanes = 0
        
        if "oneway" in edge_attr:
            oneway = int(edge_attr["oneway"])
        else:
            oneway = 0
            
        if "maxspeed" in edge_attr:
            maxspeed = int(edge_attr["maxspeed"].split()[0])
        else:
            maxspeed = 0
        edge_attributes.append([length, bridge, tunnel, lanes, oneway, maxspeed])
    edge_attributes = np.array(edge_attributes)
    edge_attr = torch.tensor(edge_attributes, dtype=torch.float)
    edge_index = torch.tensor([sources, targets], dtype=torch.long)
    data = Data(edge_index=edge_index, x=x, y=y, edge_attr=edge_attr)
    graphs.append(data)
    if tim>10:
        break
    tim+=1
print("total number:", len(graphs))

[653688, 653689, 653690, 653691, 653693, 18166124, 21300195, 21300198, 21748287, 24960494, 24960499, 60957875, 60957877, 122575957, 122575959, 122575962, 122575964, 122576018, 122576021, 122576024, 122576027, 122584862, 122584867, 122584877, 122584881, 122584885, 122584894, 122584913, 122584915, 122589747, 122589753, 122589759, 122589764, 122589767, 122603065, 122603067, 122603070, 122603075, 122603077, 122603085, 122603088, 122603101, 122617564, 122617576, 122617579, 122617618, 122617621, 122617624, 122617627, 122617632, 122617642, 122624727, 122624730, 122624732, 122624774, 122624776, 122624778, 122624822, 122624825, 122624841, 122624843, 122624852, 122624855, 122636575, 122636581, 122636588, 122639292, 122639295, 122639298, 122649328, 122649330, 122649336, 122649338, 122649345, 122652017, 122655341, 122655345, 122655351, 122655709, 122655712, 122655717, 122655724, 122655726, 122655732, 122655743, 122655747, 122659929, 122659932, 122663749, 122668890, 122668894, 122668897, 122693301,

AttributeError: 'list' object has no attribute 'split'

In [ ]:
from torch_geometric.nn import GraphConv, GCNConv, SAGEConv, GATConv
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F

class GNNModel(nn.Module):
    def __init__(self, classes):
        super(GNNModel, self).__init__()
        self.conv1 = GATConv(3, 64)
        self.conv2 = GATConv(64, 64)
        self.conv3 = GATConv(64, 64)
        self.linear = Linear(64, classes)
    
    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, 0.4)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, 0.4)
        x = self.conv3(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, 0.4)
        x = self.linear(x)
        return x

In [ ]:
from torch_geometric.data import DataLoader
from torch.optim import Adam, AdamW

batch_size = 8
train_sets = graphs[:-10]
test_sets = graphs[-10:]
train_loader = DataLoader(train_sets, batch_size=batch_size)
model = GNNModel(2)
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=5e-5)
criterion = nn.CrossEntropyLoss()
best_loss = float("inf")
epochs = 201
interval = 50

for epoch in range(epochs):
    model.train()
    losses = 0
    nums = len(train_sets)
    for _, data in enumerate(train_loader):
        data = data
        labels = data.y
        predicts = model(data)
        loss = criterion(predicts, labels)
        losses+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"epoch:{epoch}, loss:{losses/nums}")
    avg_loss = losses/nums
    if avg_loss<best_loss:
        best_loss = avg_loss
    if epoch%interval == 0:
        model.eval()
        acc_avg = 0
        times = 0
        for _, data in enumerate(test_sets):
            _, pred = model(data).max(dim=1)
            correct = int(pred.eq(data.y).sum().item())
            acc = correct / int(data.x.shape[0])
            acc_avg+=acc
            times+=1
        print(f"acc: {acc_avg/times}")